In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import keras

In [5]:
model = keras.models.load_model('checkpoint/style_model_5.keras', compile=False)
model.trainable = False
model(tf.random.normal((2, 1, 512, 512, 3)))
model.summary()

Model: "style_transfer_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ada_in_2 (AdaIN)            multiple                  0         
                                                                 
 encoder_2 (Encoder)         multiple                  3505728   
                                                                 
 decoder_2 (Decoder)         multiple                  3505219   
                                                                 
Total params: 7010955 (26.74 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 7010955 (26.74 MB)
_________________________________________________________________


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converted = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(converted)

INFO:tensorflow:Assets written to: /tmp/tmpjw09qxjb/assets


INFO:tensorflow:Assets written to: /tmp/tmpjw09qxjb/assets
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 31, Total Ops 135, % non-converted = 22.96 %
 * 31 ARITH ops

- arith.constant:   31 occurrences  (f32: 20, i32: 11)



  (f32: 3)
  (f32: 27)
  (f32: 6)
  (f32: 4)
  (f32: 27)
  (f32: 2)
  (uq_8: 18)
  (f32: 3)
  (f32: 2)
  (f32: 1)
  (f32: 1)
  (f32: 2)
  (f32: 2)
  (f32: 3)
